In [1]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Paso 1: Leer los datos geoespaciales
# Manzanas
manzanas_shp = '../data/manzanas/090030001m.shp'
gdf_manzanas = gpd.read_file(manzanas_shp)[['IDENTIFICA', 'geometry']]

# Polígono de Coyoacán
coyoacan_polygon = gpd.read_file('../data/limites/poligonos_alcaldias_cdmx.shp')
coyoacan_polygon = coyoacan_polygon[coyoacan_polygon['NOMGEO'] == 'Coyoacán']

# Uso de suelo
shapefile_path = '../data/uso_suelo/uso-de-suelo.shp'
gdf_uso_suelo = gpd.read_file(shapefile_path, encoding='latin1')[['us_dscr', 'geometry']]

# Eliminar registros con NaN en 'us_dscr'
gdf_uso_suelo = gdf_uso_suelo.dropna(subset=['us_dscr'])

# Alinear CRS
common_crs = coyoacan_polygon.crs
gdf_manzanas = gdf_manzanas.to_crs(common_crs)
gdf_uso_suelo = gdf_uso_suelo.to_crs(common_crs)

# Recortar al área de Coyoacán
gdf_manzanas_coyoacan = gpd.clip(gdf_manzanas, coyoacan_polygon)
gdf_uso_suelo_coyoacan = gpd.clip(gdf_uso_suelo, coyoacan_polygon)

# Unión espacial
gdf_union = gpd.sjoin(
    gdf_manzanas_coyoacan,
    gdf_uso_suelo_coyoacan,
    how='left',
    predicate='intersects'
)[['IDENTIFICA', 'us_dscr']]

# Eliminar filas con NaN en 'us_dscr' después de la unión
gdf_union = gdf_union.dropna(subset=['us_dscr'])

# Determinar uso de suelo predominante
uso_suelo_por_manzana = gdf_union.groupby(['IDENTIFICA', 'us_dscr'])\
                                 .size()\
                                 .reset_index(name='counts')

uso_suelo_predominante = uso_suelo_por_manzana.sort_values('counts', ascending=False)\
                                              .drop_duplicates('IDENTIFICA')

# Unir el uso de suelo predominante a las manzanas
gdf_manzanas_coyoacan = gdf_manzanas_coyoacan.merge(
    uso_suelo_predominante[['IDENTIFICA', 'us_dscr']],
    on='IDENTIFICA',
    how='left'
)

# Asignar 'Sin Datos' a manzanas sin información de uso de suelo
gdf_manzanas_coyoacan['us_dscr'] = gdf_manzanas_coyoacan['us_dscr'].fillna('Sin Datos')

# Simplificar geometrías (opcional)
gdf_manzanas_coyoacan['geometry'] = gdf_manzanas_coyoacan['geometry'].simplify(
    tolerance=0.0001, preserve_topology=True
)

# Preparar la paleta de colores
tipos_uso_suelo = gdf_manzanas_coyoacan['us_dscr'].unique()
colores = [
    '#e41a1c', '#377eb8', '#4daf4a', '#984ea3',
    '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999',
    '#a6cee3', '#1f78b4', '#b2df8a', '#33a02c',
    '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00',
    '#cab2d6', '#6a3d9a', '#ffff99'
] * 10  # Multiplicar para asegurar suficientes colores

color_dict = dict(zip(tipos_uso_suelo, colores[:len(tipos_uso_suelo)]))

# Crear el mapa
centro = gdf_manzanas_coyoacan.unary_union.centroid
m = folium.Map(location=[centro.y, centro.x], zoom_start=13)

def style_function(feature):
    uso = feature['properties']['us_dscr']
    return {
        'fillColor': color_dict.get(uso, 'gray'),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    }

folium.GeoJson(
    gdf_manzanas_coyoacan,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['us_dscr'], aliases=['Uso de Suelo:'])
).add_to(m)

# Añadir la leyenda
def generate_legend_html(color_dict):
    html = '''
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        width: 300px;
        background-color: white;
        border:2px solid grey;
        z-index:9999;
        font-size:12px;
        padding: 10px;
        max-height: 300px;
        overflow-y: auto;
        ">
        <b>Usos de Suelo</b><br>
    '''
    for uso, color in color_dict.items():
        html += f'<i style="background:{color};width:12px;height:12px;float:left;margin-right:8px;"></i>{uso}<br>'
    html += '</div>'
    return html

legend_html = generate_legend_html(color_dict)
m.get_root().html.add_child(folium.Element(legend_html))

# Ajustar el mapa a los límites de los datos
bounds = gdf_manzanas_coyoacan.total_bounds  # [minx, miny, maxx, maxy]
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

# Mostrar el mapa
m


/tmp/ipykernel_95906/1745204720.py:78: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centro = gdf_manzanas_coyoacan.unary_union.centroid


In [4]:
manzanas_shp = '../data/manzanas/090030001m.shp'
gdf_manzanas = gpd.read_file(manzanas_shp)

In [5]:
gdf_manzanas

,IDENTIFICA,CVEGEO,geometry
0,1,0900300011497015,"POLYGON ((2803618.272 816084.655, 2803629.084 ..."
1,2,0900300010713073,"POLYGON ((2800050.629 817743.071, 2800047.704 ..."
2,3,0900300010855034,"POLYGON ((2799116.786 816540.751, 2799115.55 8..."
3,4,0900300011251001,"POLYGON ((2802183.591 818100.751, 2802195.869 ..."
4,5,0900300011251026,"POLYGON ((2802247.866 818010.814, 2802247.655 ..."
...,...,...,...
4808,4809,0900300011548024,"POLYGON ((2796078.568 815297.134, 2796077.364 ..."
4809,4810,0900300011336014,"POLYGON ((2802896.383 816945.221, 2802906.594 ..."
4810,4811,0900300011660030,"POLYGON ((2797303.256 814730.717, 2797305.219 ..."
4811,4812,090030001150A023,"POLYGON ((2803218.608 815809.155, 2803207.165 ..."


## Queremos ver como estan relaciondas las colonias y las manzanas con el CVEGEO

In [6]:
ageb_shp = '~/Downloads/poligono_ageb_urbanas_cdmx/poligono_ageb_urbanas_cdmx.shp'
gdf_ageb = gpd.read_file(ageb_shp)

In [7]:
print(gdf_ageb.head())

          CVEGEO CVE_ENT CVE_MUN CVE_LOC CVE_AGEB  \
0  0901000011716      09     010    0001     1716   
1  0901000012150      09     010    0001     2150   
2  0901000011133      09     010    0001     1133   
3  0901000011307      09     010    0001     1307   
4  0901000010281      09     010    0001     0281   

                                            geometry  
0  POLYGON ((-99.25882 19.32558, -99.25834 19.325...  
1  POLYGON ((-99.1917 19.37893, -99.1917 19.37879...  
2  POLYGON ((-99.1776 19.35182, -99.17766 19.3517...  
3  POLYGON ((-99.20805 19.31277, -99.20768 19.312...  
4  POLYGON ((-99.24228 19.38451, -99.24233 19.384...  


In [9]:
print(gdf_ageb.columns)
print(gdf_ageb.crs)
print(gdf_ageb.geometry)

Index(['CVEGEO', 'CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB', 'geometry'], dtype='object')
EPSG:4326
0       POLYGON ((-99.25882 19.32558, -99.25834 19.325...
1       POLYGON ((-99.1917 19.37893, -99.1917 19.37879...
2       POLYGON ((-99.1776 19.35182, -99.17766 19.3517...
3       POLYGON ((-99.20805 19.31277, -99.20768 19.312...
4       POLYGON ((-99.24228 19.38451, -99.24233 19.384...
                              ...                        
2426    POLYGON ((-99.00253 19.35744, -99.00255 19.355...
2427    POLYGON ((-98.99932 19.35467, -98.99889 19.354...
2428    POLYGON ((-98.98392 19.34561, -98.98367 19.345...
2429    POLYGON ((-99.06016 19.36342, -99.06042 19.362...
2430    POLYGON ((-99.0568 19.36313, -99.0576 19.35951...
Name: geometry, Length: 2431, dtype: geometry


In [11]:
# Verifica que ambos GeoDataFrames tengan el mismo CRS
gdf_manzanas = gdf_manzanas.to_crs(gdf_ageb.crs)

# Realiza un cruce espacial
gdf_manzanas_ageb = gpd.sjoin(gdf_manzanas, 
                              gdf_ageb, 
                              how="left", 
                              predicate="intersects")

# Visualiza los datos combinados
print(gdf_manzanas_ageb.head())

   IDENTIFICA       CVEGEO_left  \
0           1  0900300011497015   
1           2  0900300010713073   
2           3  0900300010855034   
3           4  0900300011251001   
4           5  0900300011251026   

                                            geometry  index_right  \
0  POLYGON ((-99.10271 19.31447, -99.10261 19.314...         1946   
1  POLYGON ((-99.13643 19.33014, -99.13646 19.330...         1845   
2  POLYGON ((-99.14557 19.31942, -99.14559 19.319...         1879   
3  POLYGON ((-99.11601 19.33299, -99.1159 19.3329...         1921   
4  POLYGON ((-99.11542 19.33216, -99.11542 19.332...         1921   

    CVEGEO_right CVE_ENT CVE_MUN CVE_LOC CVE_AGEB  
0  0900300011497      09     003    0001     1497  
1  0900300010713      09     003    0001     0713  
2  0900300010855      09     003    0001     0855  
3  0900300011251      09     003    0001     1251  
4  0900300011251      09     003    0001     1251  


In [12]:
import geopandas as gpd

# Lista de shapefiles con rutas completas
shapefiles = [
    "~/Downloads/09/DISTRITO.shp",
    "~/Downloads/09/ENTIDAD.shp",
    "~/Downloads/09/MUNICIPIO.shp",
    "~/Downloads/09/SECCION.shp"
]

# Explorar cada shapefile
for shp in shapefiles:
    print(f"Explorando {shp}...")
    try:
        gdf = gpd.read_file(shp)
        print("Columnas disponibles:", gdf.columns)
        print("Primeras filas:\n", gdf.head(), "\n")
        print("Sistema de referencia (CRS):", gdf.crs, "\n")
    except Exception as e:
        print(f"Error leyendo {shp}: {e}\n")

Explorando ~/Downloads/09/DISTRITO.shp...
Columnas disponibles: Index(['gid', 'id', 'entidad', 'distrito', 'tipo', 'control', 'geometry1_',
       'geometry'],
      dtype='object')
Primeras filas:
    gid   id  entidad  distrito  tipo  control geometry1_  \
0    1  1.0        9         1     6      0.0       None   
1    2  2.0        9         2     7      0.0       None   
2    3  3.0        9         3     6      0.0       None   
3    4  4.0        9         4     6      0.0       None   
4    5  5.0        9         5     7      0.0       None   

                                            geometry  
0  POLYGON ((486884.035 2166534.968, 487088.745 2...  
1  POLYGON ((486955.834 2158280.658, 486971.543 2...  
2  POLYGON ((478505.353 2157846.004, 478520.669 2...  
3  POLYGON ((491075.959 2138646.711, 491305.972 2...  
4  POLYGON ((485837.484 2134940.874, 486046.859 2...   

Sistema de referencia (CRS): EPSG:32614 

Explorando ~/Downloads/09/ENTIDAD.shp...
Columnas disponibles: Ind